In [1]:
import numpy as np
from qdrant_client import QdrantClient
from qdrant_client.http.models import PointStruct
import pandas as pd
from qdrant_interface import QDrantInterface


clustered_vectors = pd.read_csv('./data/clustered_vectors.csv').values

qdrant = QdrantClient(":memory:")
collection_name = "vector_collection"

qdrant.recreate_collection(
    collection_name=collection_name,
    vectors_config={"size": clustered_vectors.shape[1], "distance": "Cosine"}
)

# Upload the vectors
points = [
    PointStruct(id=i, vector=vector.tolist())
    for i, vector in enumerate(clustered_vectors)
]

qdrant.upsert(collection_name=collection_name, points=points)

# source https://python-client.qdrant.tech/

C:\Users\dhimi\AppData\Local\Temp\ipykernel_35628\2033497685.py:11: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [2]:
# doing a search query test
query_vector = clustered_vectors[0].tolist()

search_result = qdrant.search(
    collection_name=collection_name,
    query_vector=query_vector,
    limit=10 
)

# Print the search results
for result in search_result:
    print(result)

id=0 version=0 score=0.9999999988241883 payload={} vector=None shard_key=None order_value=None
id=4791 version=0 score=0.9733456684059761 payload={} vector=None shard_key=None order_value=None
id=112 version=0 score=0.9731442626214278 payload={} vector=None shard_key=None order_value=None
id=596 version=0 score=0.9729512723259301 payload={} vector=None shard_key=None order_value=None
id=4477 version=0 score=0.97288293892145 payload={} vector=None shard_key=None order_value=None
id=6595 version=0 score=0.9728680538292809 payload={} vector=None shard_key=None order_value=None
id=4060 version=0 score=0.9728547648246987 payload={} vector=None shard_key=None order_value=None
id=9043 version=0 score=0.9728345841123753 payload={} vector=None shard_key=None order_value=None
id=1091 version=0 score=0.9728207069369069 payload={} vector=None shard_key=None order_value=None
id=2757 version=0 score=0.9727718166003076 payload={} vector=None shard_key=None order_value=None


In [3]:
import os

csv_file_path = './data/clustered_vectors.csv'
org_size = os.path.getsize(csv_file_path)

print(f"Original vector dataset size: {org_size} bytes")


Original vector dataset size: 193002578 bytes


In [5]:
import numpy as np
from qdrant_client import QdrantClient
from qdrant_client.http.models import PointStruct
import pandas as pd

# Init qdrant client like the test before

clustered_vectors = pd.read_csv('./data/clustered_vectors.csv').values

qdrant = QdrantClient(":memory:")
collection_name = "vector_collection"

qdrant.recreate_collection(
    collection_name=collection_name,
    vectors_config={"size": clustered_vectors.shape[1], "distance": "Cosine"}
)

# Upload the vectors
points = [
    PointStruct(id=i, vector=vector.tolist())
    for i, vector in enumerate(clustered_vectors)
]

qdrant.upsert(collection_name=collection_name, points=points)
#*******************************************************************************
# do an einitial estimation of the size per vector
size_per_vector = clustered_vectors.shape[1] * 4  # Each float is 4 bytes

num_vectors = clustered_vectors.shape[0]

# an stimation of the total storage size
estimated_qdrant_size = size_per_vector * num_vectors

print(f"Estimated size of data in Qdrant: {estimated_qdrant_size} bytes")

C:\Users\dhimi\AppData\Local\Temp\ipykernel_35628\1281254356.py:13: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


Estimated size of data in Qdrant: 40000000 bytes


In [7]:
org_size = os.path.getsize(csv_file_path)

print(f"Original size: {org_size} bytes")
print(f"Estimated size  in Qdrant: {estimated_qdrant_size} bytes")

compression_ratio = org_size / estimated_qdrant_size
print(f"Compression ratio (original/Qdrant): {compression_ratio:.2f}")

Original vector dataset size: 193002578 bytes
Estimated size of data in Qdrant: 40000000 bytes
Compression ratio (original/Qdrant): 4.83


In [4]:
import numpy as np
import pandas as pd
import os
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PointStruct

clustered_vectors = pd.read_csv('./data/clustered_vectors.csv').values
qdrant = QdrantClient(path="./qdrant_data")
collection_name = "vector_collection"
qdrant.create_collection(
   collection_name=collection_name,
   vectors_config=VectorParams(size=clustered_vectors.shape[1], distance=Distance.COSINE),
)
points = [
    PointStruct(id=i, vector=vector.tolist())
    for i, vector in enumerate(clustered_vectors)
]

qdrant.upsert(collection_name=collection_name, points=points)
# *****************************************************

def get_directory_size(directory):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(directory):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            total_size += os.path.getsize(fp)
    return total_size

# Getting thr qdrant data directory size
qdrant_data_size = get_directory_size('./qdrant_data/collection/vector_collection')
print(f"Size of data in Qdrant: {qdrant_data_size} bytes")

original_size = os.path.getsize('./data/clustered_vectors.csv')
print(f"Original vector dataset size: {original_size} bytes")

# Calculate the compression ratio
compression_ratio = original_size / qdrant_data_size
print(f"Compression ratio (original/Qdrant): {compression_ratio:.2f}")

#https://www.slingacademy.com/article/python-calculating-total-size-of-a-folder-and-its-contents/


Size of data in Qdrant: 95928320 bytes
Original vector dataset size: 192957645 bytes
Compression ratio (original/Qdrant): 2.01


In [2]:

collection_name = "vector_collection"
r = qdrant.get_collection(collection_name)
print(r)

status=<CollectionStatus.GREEN: 'green'> optimizer_status=<OptimizersStatusOneOf.OK: 'ok'> vectors_count=None indexed_vectors_count=0 points_count=10000 segments_count=1 config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=1000, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=None), shard_number=None, sharding_method=None, replication_factor=None, write_consistency_factor=None, read_fan_out_factor=None, on_disk_payload=None, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=None, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=1), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quantization_con

In [3]:
# to be done



    def get_rows_cnt(self, table_name):
        query = f'SELECT COUNT(*) FROM {table_name}'
        result = self.conn.execute(query).fetchall()
        # print(result)
        self.conn.commit()
        return result[0][0]

    def similarity_search(self, table_name, embedding_vector, metrix):
        if metrix == "l2":
            symbol = "<->"
        elif metrix == "cosine":
            symbol = "<=>"
        else:
            print("Error with matrix type")
            return
        sim_query = f"""
        SELECT id, embedding {symbol} (%s) AS distance
        FROM {table_name}
        ORDER BY distance ASC
        LIMIT 5
        """
        result = self.conn.execute(sim_query, (embedding_vector,)).fetchall()
        # print(result)
        return result


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 10)

In [ ]:
# test for insert single vector
qdrant_interface = QDrantInterface(data_path='./qdrant_data')
print("Connected to Qdrant server.")

qdrant_interface.drop_table('vector_collection')

qdrant_interface.create_table(collection_name='vector_collection', vector_size=5)
print("Collection created successfully.")

# Insert a single vector
try:
    sample_vector = np.array([0.1, 0.2, 0.3, 0.4, 0.5])
    qdrant_interface.insert_single_vector(collection_name='vector_collection', vector=sample_vector)
    print("Vector inserted successfully.")
except Exception as e:
    print(f"An error occurred while inserting the vector: {e}")

qdrant_interface.disconnect_server()
print("Disconnected from Qdrant server.")

In [9]:
import numpy as np
from qdrant_client import QdrantClient
from qdrant_client.http.models import PointStruct
import pandas as pd
from qdrant_interface import QDrantInterface

# testing
qdrant_interface = QDrantInterface(data_path='./qdrant_data')
print("Connected to Qdrant server.")

qdrant_interface.drop_table('vector_collection')

qdrant_interface.create_table(collection_name='vector_collection', vector_size=1000)
print("Collection created successfully.")

# # Insert a single vector
# try:
#     sample_vector = np.array([0.1, 0.2, 0.3, 0.4, 0.5])
#     qdrant_interface.insert_single_vector(collection_name='vector_collection', vector=sample_vector)
#     print("Vector inserted successfully.")
# except Exception as e:
#     print(f"An error occurred while inserting the vector: {e}")

# Insert vectors from CSV
try:
    points = qdrant_interface.transfer_csv('./data/clustered_vectors.csv')
    qdrant_interface.insert_vector_from_csv(collection_name='vector_collection', points=points)
    print("Vectors inserted successfully from CSV.")
except Exception as e:
    print(f"An error occurred while inserting vectors from CSV: {e}")

# Get row count
try:
    rows_count = qdrant_interface.get_rows_cnt(collection_name='vector_collection')
    print(f"Number of vectors in the collection: {rows_count}")
except Exception as e:
    print(f"An error occurred while getting the row count: {e}")

try:
    sample_vector = [0.1, 0.2, 0.3, 0.4, 0.5] * 200  # Adjusted to match 1000 dimensions
    search_result = qdrant_interface.similarity_search(collection_name='vector_collection', embedding_vector=sample_vector, metric='cosine', limit=5)
    print(f"Similarity search result: {search_result}")
except Exception as e:
    print(f"An error occurred during similarity search: {e}")

qdrant_interface.disconnect_server()
print("Disconnected from Qdrant server.")


Connected to Qdrant server.
Collection created successfully.
Vectors inserted successfully from CSV.
An error occurred while getting the row count: 'CollectionInfo' object is not subscriptable
Similarity search result: [{'id': 8484, 'score': 0.7967769209702504}, {'id': 6689, 'score': 0.7963385296990608}, {'id': 1685, 'score': 0.7961955815548707}, {'id': 170, 'score': 0.7957872332440298}, {'id': 503, 'score': 0.7956461530576955}]
Disconnected from Qdrant server.


In [2]:
import numpy as np
from qdrant_client import QdrantClient
from qdrant_client.http.models import PointStruct
import pandas as pd
from qdrant_interface import QDrantInterface

# testing
qdrant_interface = QDrantInterface(data_path='./qdrant_data')
print("Connected to Qdrant server.")

qdrant_interface.drop_table('vector_collection')

qdrant_interface.create_table(collection_name='vector_collection', vector_size=1000, index_types="Hnsw")
print("Collection created successfully.")

# # Insert a single vector
# try:
#     sample_vector = np.array([0.1, 0.2, 0.3, 0.4, 0.5])
#     qdrant_interface.insert_single_vector(collection_name='vector_collection', vector=sample_vector)
#     print("Vector inserted successfully.")
# except Exception as e:
#     print(f"An error occurred while inserting the vector: {e}")

# Insert vectors from CSV
try:
    points = qdrant_interface.transfer_csv('./data/clustered_vectors.csv')
    qdrant_interface.insert_vector_from_csv(collection_name='vector_collection', points=points)
    print("Vectors inserted successfully from CSV.")
except Exception as e:
    print(f"An error occurred while inserting vectors from CSV: {e}")

# Get row count
try:
    rows_count = qdrant_interface.get_rows_cnt(collection_name='vector_collection')
    print(f"Number of vectors in the collection: {rows_count}")
except Exception as e:
    print(f"An error occurred while getting the row count: {e}")

# try:
#     sample_vector = [0.1, 0.2, 0.3, 0.4, 0.5]   # Adjusted to match 1000 dimensions
#     search_result = qdrant_interface.similarity_search(collection_name='vector_collection', embedding_vector=sample_vector, metric='cosine', limit=5)
#     print(f"Similarity search result: {search_result}")
# except Exception as e:
#     print(f"An error occurred during similarity search: {e}")

qdrant_interface.disconnect_server()
print("Disconnected from Qdrant server.")


Connected to Qdrant server.
Collection created successfully.
Vectors inserted successfully from CSV.
status=<CollectionStatus.GREEN: 'green'> optimizer_status=<OptimizersStatusOneOf.OK: 'ok'> vectors_count=None indexed_vectors_count=0 points_count=10000 segments_count=1 config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=1000, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=None), shard_number=None, sharding_method=None, replication_factor=None, write_consistency_factor=None, read_fan_out_factor=None, on_disk_payload=None, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=None, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_opti